## Panel - Interacting with the visual

In the [previous notebook](Heat_and_Trees_part_1.ipynb) we generated a visual of the thresholded surface temperature over Philadelphia. This plot is rendered with bokeh and uses a running python kernel to resample the data when zooming in and out. But what if we want to allow the user to set their own threshold. To enable this kind of interactivity we'll use [panel](https://panel.pyviz.org).

In [ ]:
import intake
import geopandas as gpd

import holoviews as hv
from holoviews import opts
import hvplot.xarray
import hvplot.pandas

import panel as pn

from geoviews.tile_sources import EsriImagery

hv.extension('bokeh')

### Load the data

First load in the data that we computed and saved in the previous notebook. We'll persist it in memory for easy access.

In [ ]:
mtf_source = intake.open_zarr('data/mean_temp.zarr')
mean_temp_f = mtf_source.to_dask()
mean_temp_f.persist()

Make the same special colormap:

In [ ]:
import colorcet as cc
special_cmap = cc.fire[::-1][90:]

### Use panel.interact

Write a function that generates the thresholded plot and takes the temperature at which to threshold as an argument. Use the `pn.interact` decorator to "widgetize" the function. 

In [ ]:
@pn.interact(thresh=(70, 100))
def threshold(thresh=90):
    thresholded_temp_p = (mean_temp_f.where(mean_temp_f > thresh)
        .hvplot(x='x', y='y', cmap=special_cmap, crs=crs, height=500, width=450, colorbar=False, legend=False)
        .relabel(f'Mean Temp (F) > {thresh}')
        .redim(value='Temperature (F)'))

    return thresholded_temp_p + thresholded_temp_p.options(alpha=.3) * EsriImagery

In [ ]:
threshold

There are a few limitations to this approach. For instance, this approach: 
  1. Recomputes the data every time there is an interaction
  2. Re-renders the whole plot every time
  2. Depends on a python kernel being present - doesn't work in a static version
  
### Using color clipping

To combat some of these limitations we can use color clipping and some recently developed panel functionality to link interactions using js. 

We'll start by defining the plot.

In [ ]:
p = (mean_temp_f
    .hvplot(x='x', y='y', geo=True, height=500, width=550, cmap='rainbow', alpha=.5, legend=False)
    .relabel('Mean Surface Temp (F)'))

Next we'll set up a clipped colormap that only shows values between an upper and lower bound.

In [ ]:
options = opts.Image(clipping_colors={'min': 'transparent', 'max': 'transparent'}, 
                     cmap=special_cmap)
clipped = p.opts(options, clone=True)
clipped.redim.range(value=(90, 120))

clipped

So we've solved the problem of the data being recomputed every time, so now we can add in interaction. 

### Using JS linking

Now we are trying to interact with things that are on the javascript side. So instead of having the interaction happen on the python side, we can have it on the js side. This will make it faster and also allow it to happen *without a python kernel running*.

In [ ]:
start, end = (int(mean_temp_f.temperature.min() - 1), 
              int(mean_temp_f.temperature.max() + 1))

slider = pn.widgets.RangeSlider(start=start, end=end, value=(start, end), 
                                step=1, bar_color='#ff0000')

slider.jslink(clipped, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
"""})

pn.Row(slider, clipped * EsriImagery)

### Adding some more interaction

To round out the dashboard, we can add a base map selector, an alpha slider, a tree toggle...

In [ ]:
from holoviews.streams import Params

import geoviews as gv
from geoviews.tile_sources import tile_sources

In [ ]:
slider = pn.widgets.RangeSlider(start=start, end=end, value=(start, end), 
                                step=1, bar_color='#ff0000')

slider.jslink(clipped, code={'value': """
color_mapper.low = source.value[0];
color_mapper.high = source.value[1];
"""})

selector = pn.widgets.Select(name='basemap', value=EsriImagery, options=tile_sources)

def set_tile_source(value):
    return value
    
dmap = hv.DynamicMap(set_tile_source, streams=[Params(selector, ['value'])])

pn.Row(
    pn.Column(
        slider, 
        selector),
    clipped * dmap)

> To Do: Add an alpha slider, toggle off basemap, toggle off trees, change dynspread of trees

## Adding in the Street Tree data

OpenDataPhilly released an inventory of all the street trees in the city. Street trees are trees that are planted along streets, not those in parks and private property. Using `datashader` we can easily plot these 100,000 points.


It is hypothesized that where there are more trees the land surface temperature will be less extreme. To explore this, we will overlay street trees with the thresholded land surface temperature:

In [ ]:
url = 'http://data.phl.opendata.arcgis.com/datasets/957f032f9c874327a1ad800abd887d17_0.geojson'
trees = gpd.read_file(url)

In [ ]:
tree_p = trees.hvplot(geo=True, datashade=True, height=500, width=500, 
                      legend=False, dynspread=30).relabel('Street Tree Density')

thresholded_temp_p.options(alpha=.5) * tree_p.options(alpha=.5)

In [ ]:
tree_p = trees.hvplot(geo=True, datashade=True, height=500, width=500, 
                      legend=False, dynspread=30).relabel('Street Tree Density')